In [ ]:
!wget -nc http://snap.stanford.edu/data/amazon/productGraph/categoryFiles/reviews_Electronics_5.json.gz
!wget -nc http://snap.stanford.edu/data/amazon/productGraph/categoryFiles/meta_Electronics.json.gz
!gzip -d reviews_Electronics_5.json.gz
!gzip -d meta_Electronics.json.gz

--2020-12-11 05:15:02--  http://snap.stanford.edu/data/amazon/productGraph/categoryFiles/reviews_Electronics_5.json.gz
Resolving snap.stanford.edu (snap.stanford.edu)... 171.64.75.80
Connecting to snap.stanford.edu (snap.stanford.edu)|171.64.75.80|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 495854086 (473M) [application/x-gzip]
Saving to: ‘reviews_Electronics_5.json.gz’

             review  26%[====>               ] 125.41M  6.49MB/s    eta 25s    ^C
--2020-12-11 05:15:11--  http://snap.stanford.edu/data/amazon/productGraph/categoryFiles/meta_Electronics.json.gz
Resolving snap.stanford.edu (snap.stanford.edu)... 171.64.75.80
Connecting to snap.stanford.edu (snap.stanford.edu)|171.64.75.80|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 186594679 (178M) [application/x-gzip]
Saving to: ‘meta_Electronics.json.gz’

meta_Electronics.js   4%[                    ]   8.68M  8.41MB/s               ^C
gzip: reviews_Electronics_5.json already

In [1]:
import pandas as pd
import json
from gensim import corpora, models, similarities
from nltk.corpus import stopwords
from datetime import timedelta
import re
import datetime
import nltk
import pprint
import nltk.data

import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem.lancaster import LancasterStemmer
import json
import sqlite3
import ast

from gensim import corpora, models, similarities
from nltk.corpus import stopwords
from datetime import timedelta
import re
import datetime
import nltk
import pprint
import nltk.data

import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
import json
import sqlite3
import ast

import string
import nltk
from nltk.tokenize import TreebankWordTokenizer
from nltk.stem.porter import PorterStemmer
from sklearn.feature_extraction.text import TfidfVectorizer

import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import TfidfVectorizer
from collections import Counter
from textblob import TextBlob

REMOVE_PUNCTUATION_TABLE = str.maketrans({x: None for x in string.punctuation})
TOKENIZER = TreebankWordTokenizer()
STEMMER = PorterStemmer()

In [2]:
pip install textblob

n


In [3]:
import nltk
nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [51]:
feedback = {}

Load data from dataset

main function here:

In [4]:
reviews_Musical_Instruments_5 = pd.read_json('reviews_Electronics_5.json',lines=True)
meta_list = []
with open('meta_Electronics.json') as f:
  for line in f.readlines():
    line = json.dumps(ast.literal_eval(line))
    d = json.loads(line)
    if 'title' not in d:
      continue
    else:
      title = d['title']
      product_id = d['asin']
      img_url = d['imUrl']
  
      meta = []
      
      meta.append(product_id)
      meta.append(title)
      meta.append(img_url)

      meta_list.append(meta)
column_names = ['asin','title','image_url']
meta = pd.DataFrame(meta_list, columns=column_names)
entire_merged_df = reviews_Musical_Instruments_5[['asin','overall','reviewerID','reviewText']].merge(meta[['asin', 'title','image_url']], on='asin', how='inner')
entire_merged_df['reviewText'] = entire_merged_df['reviewText'].str.lower()
asin_review_df = entire_merged_df[['asin','reviewText']]
product_dict = dict(zip(entire_merged_df.asin, entire_merged_df.title))
product_dict =  {k: v.lower() for k, v in product_dict.items()}
review_list=list(entire_merged_df['reviewText'])

In [5]:
entire_merged_df

,asin,overall,reviewerID,reviewText,title,image_url
0,0528881469,5,AO94DHGC771SJ,we got this gps for my husband who is an (otr)...,Rand McNally 528881469 7-inch Intelliroute TND...,http://ecx.images-amazon.com/images/I/51FnRkJq...
1,0528881469,1,AMO214LNFCEI4,"i'm a professional otr truck driver, and i bou...",Rand McNally 528881469 7-inch Intelliroute TND...,http://ecx.images-amazon.com/images/I/51FnRkJq...
2,0528881469,3,A3N7T0DY83Y4IG,"well, what can i say. i've had this unit in m...",Rand McNally 528881469 7-inch Intelliroute TND...,http://ecx.images-amazon.com/images/I/51FnRkJq...
3,0528881469,2,A1H8PY3QHMQQA0,"not going to write a long review, even thought...",Rand McNally 528881469 7-inch Intelliroute TND...,http://ecx.images-amazon.com/images/I/51FnRkJq...
4,0528881469,1,A24EV6RXELQZ63,i've had mine for a year and here's what we go...,Rand McNally 528881469 7-inch Intelliroute TND...,http://ecx.images-amazon.com/images/I/51FnRkJq...
...,...,...,...,...,...,...
1643681,B00LGQ6HL8,5,A34BZM6S9L7QI4,burned these in before listening to them for a...,Brainwavz S5 In Ear Headphones,http://ecx.images-amazon.com/images/I/41%2BcVY...
1643682,B00LGQ6HL8,5,A1G650TTTHEAL5,some people like dj style headphones or earbud...,Brainwavz S5 In Ear Headphones,http://ecx.images-amazon.com/images/I/41%2BcVY...
1643683,B00LGQ6HL8,5,A25C2M3QF9G7OQ,i&#8217;m a big fan of the brainwavz s1 (actua...,Brainwavz S5 In Ear Headphones,http://ecx.images-amazon.com/images/I/41%2BcVY...
1643684,B00LGQ6HL8,5,A1E1LEVQ9VQNK,"i've used thebrainwavz s1 in ear headphones, a...",Brainwavz S5 In Ear Headphones,http://ecx.images-amazon.com/images/I/41%2BcVY...


In [6]:
query = 'earphone,wireless,waterproof'

In [38]:
def parse_query(query):
  feature = "" 
  query_list = query.split(',')
  #print(query_list)
  count = 0
  for ele in query_list:  
    feature += ele
    feature += ' '   
  result_query_list = []
  result_query_list.append(query_list[0])
  result_query_list.append(feature)
  #print(feature)
  return result_query_list

In [40]:
query_list = parse_query(query)
print(query_list[0])
review_list_of_list = query_dataframe(query_list[0],entire_merged_df)
review_list_of_list
print(review_list_of_list)

review_list = get_review_list(query_list[0],entire_merged_df)
print(query_list[1])
sim_list = similarity(query_list[1], review_list_of_list)
print(sim_list)
result_sim_df = get_result_similarity_wtfeedback(sim_list)

result_max_sim_df = result_sim_df[result_sim_df.groupby('asin')['similarity'].transform('max') == result_sim_df['similarity']]
result_max_sim_df = result_max_sim_df[['asin', 'reviewText','similarity']]

#group by asin and use polarity
grouped_result_sim_df = result_sim_df.groupby(['asin'])['polarity'].sum()
grouped_result_sim_df = grouped_result_sim_df.reset_index()
grouped_result_sim_df = grouped_result_sim_df[grouped_result_sim_df.polarity > 0]

grouped_result_sim_df = grouped_result_sim_df.merge(result_max_sim_df, on='asin', how='inner')
grouped_result_sim_df


feedback = {
        'earphone wireless waterproof': [('B001GPCBPU', 1.)],
        'piano keyboard': [('B0002F52EW', 1.)]
}


result_sim_df = nn_feedback(feedback, review_list, query, grouped_result_sim_df)

result_sim_df = result_sim_df.sort_values(['similarity', 'polarity'], ascending=[False, False])
result_sim_df

earphone
[['the company that sold me the ear buds was great!  the product is awesome...very little wind noise.  they keep the sound from escaping so you can enjoy the music and eliminate most of the outside noise.  i love them!  god bless!  joe'], ["i've ordered 3 of these replacement packs for a couple of pair of shure earphones over the last couple of years.the sound isolation from the foam is excellent, they are fairly easy to use, and fall out less easily than any other option i've tried.  probably only custom-fit molding works much better.the passive sound isolation is better than active noise cancellation by manufacturers such as bose, as numerous studies have shown.  i'm always amused by the bose-users on airplanes who don't realize they would get better sound from tiny earphones like shure or etymotic than their 5lb monsters with batteries.only two minor caveats, one of which should be temporary - (1)  there are no shure earphones with the button for an iphone yet, only an adap

/usr/local/lib/python3.6/dist-packages/sklearn/feature_extraction/text.py:507: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn("The parameter 'token_pattern' will not be used"
/usr/local/lib/python3.6/dist-packages/sklearn/feature_extraction/text.py:385: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['abov', 'afterward', 'alon', 'alreadi', 'alway', 'ani', 'anoth', 'anyon', 'anyth', 'anywher', 'becam', 'becaus', 'becom', 'befor', 'besid', 'cri', 'describ', 'dure', 'els', 'elsewher', 'empti', 'everi', 'everyon', 'everyth', 'everywher', 'fifti', 'formerli', 'forti', 'ha', 'henc', 'hereaft', 'herebi', 'hi', 'howev', 'hundr', 'inde', 'latterli', 'mani', 'meanwhil', 'moreov', 'mostli', 'nobodi', 'noon', 'noth', 'nowher', 'onc', 'onli', 'otherwis', 'ourselv', 'perhap', 'pleas', 'seriou', 'sever', 'sinc', 'sincer', 'sixti', 'someon', 'someth', 'sometim', 'somewher',

,asin,polarity,reviewText,similarity
6,B00275F2GS,2.346377,buds fit well with reasonable comfort. small h...,0.235114
8,B0036J1S1W,3.835510,this three size pack fits anyone and turns unc...,0.189427
10,B004CHHLDG,0.131250,i bought this to change my headphone so that i...,0.165648
2,B001GPCBPU,2.403396,"i have many wireless headsets some good, some ...",0.129149
4,B001I1A5WS,0.402509,the sound is good for a pair of headphones/ear...,0.065408
3,B001GUPEMW,4.339785,"pros: incredibly durable, long lasting and hig...",0.058738
11,B0051RW1SQ,1.022500,this earphone case can definitely take a beati...,0.053075
14,B005J0RAI6,1.887763,xeport goal is to offer good quality earphones...,0.052474
15,B005J0W3F6,1.135130,these xeport 3010-2 earphones are perhaps the ...,0.052008
9,B003DW4A76,0.968765,"i have no reason not to give it 5 stars, it's ...",0.046367


In [37]:
result_sim_df

,asin,polarity,reviewText,similarity
0,B001GPCBPU,0.293452,"i have many wireless headsets some good, some ...",0.132372
1,B00275F2GS,0.599265,buds fit well with reasonable comfort. small h...,0.225162
2,B0036J1S1W,0.827083,this three size pack fits anyone and turns unc...,0.194154
3,B004CHHLDG,0.131250,i bought this to change my headphone so that i...,0.169781
4,B005IZ65G0,0.216632,these days i listen to music on my iphone prim...,0.026513
5,B005J0RAI6,0.186048,these days i listen to music on my iphone prim...,0.028259
6,B005J0W3F6,0.219388,these days i listen to music on my iphone prim...,0.027922


In [50]:
feedback = update_cache((12345,1), 'earphone,wireless,waterproof')

{'earphone wireless waterproof ': [('B001GPCBPU', 1.0), (12345, 1)],
 'piano keyboard ': [('B0002F52EW', 1.0)]}

In [49]:
def update_cache(asin_rel_tuple, query, feedback):
  result_query_list = parse_query(query)
  query = result_query_list[1]

  if query in feedback:
    feedback[query].append(asin_rel_tuple)
  else:
    feedback[query] = asin_rel_tuple

  return feedback

Get queried dataset and the list of reviewText lists.

In [8]:
def get_review_list(query,entire_merged_df):
  queried_df = entire_merged_df[entire_merged_df['title'].str.contains(query)]
  review_list = list(queried_df['reviewText'])
  return review_list

In [9]:
def query_dataframe(query,entire_merged_df):
  queried_df = entire_merged_df[entire_merged_df['title'].str.contains(query)]
  #print(queried_df)
  review_list = list(queried_df['reviewText'])
  #print(review_list)
  review_list_of_list = [[x] for x in review_list]

  return review_list_of_list

Compute similarity and data preprocessing.


In [10]:
def similarity(query, input):

	valid_words = getdata([query])
  
	texts = []
	for i in input:
		texts.append(getdata(i))

	dictionary = corpora.Dictionary(texts)

	new_xs = dictionary.doc2bow(valid_words)
	corpus = [dictionary.doc2bow(text) for text in texts]
	  
	tfidf = models.TfidfModel(corpus)
	featurenum = len(dictionary.token2id.keys())
	index = similarities.SparseMatrixSimilarity(tfidf[corpus],
	                                          num_features=featurenum)
	sim = index[tfidf[new_xs]]
	sim_list = sim.tolist()
	#print(len(sim_list))
  # print(len(sim_list))
	return sim_list

In [11]:
def getdata(texts):
	textTokenized = [[word.lower() for word in word_tokenize(text)] for
                    text in texts]

	englishStopwords = stopwords.words('english')

	textFilterStopwords = [
      [word for word in text if not word in englishStopwords] for text in
      textTokenized]

	englishPunctuations = [',', '.', ':', ';', '?', '(', ')', '[', ']',
                          '&', '!', '*', '@', '#', '$', '%', '<', '>',
                          '=', '{', '}', '+', '"', '-', '/', "'", '"']
	text_no_pun = [
      [word for word in text if word not in englishPunctuations]
      for text in textFilterStopwords]

	st = PorterStemmer()
	textStemed = [[st.stem(word) for word in text] for text in text_no_pun]
	flattened = [val for sublist in textStemed for val in sublist]
	return flattened

Get the result similarity dataframe without feedback.

In [28]:
def get_result_similarity_wtfeedback(sim_list):
  result_index_list = [i for i, e in enumerate(sim_list) if e != 0]
  result_sim_dict = {}
  for index in result_index_list:
    result_sim_dict[review_list_of_list[index][0]] = sim_list[index]
  result_review_list = []
  for index in result_index_list:
    result_review_list.append(review_list_of_list[index])
  result_sim_df = pd.DataFrame(list(result_sim_dict.items()),columns = ['reviewText','similarity']) 

  result_df = pd.DataFrame()
  for review in result_review_list:
    result_df = result_df.append(asin_review_df.loc[asin_review_df['reviewText'] == review[0]], ignore_index=True)
  
  pol = lambda x: TextBlob(x).sentiment.polarity
  result_df['polarity'] = result_df['reviewText'].apply(pol)

  result_sim_df = result_sim_df.merge(result_df, on='reviewText', how='inner')

  return result_sim_df

Add nearest neighbor feedback. 

In [13]:
def tokenize_and_stem(s):
    return [STEMMER.stem(t) for t 
            in TOKENIZER.tokenize(s.translate(REMOVE_PUNCTUATION_TABLE))]

In [31]:
def nn_feedback(feedback, review_list, query, result_sim_df):
  # print('*******')
  # print(result_sim_df)
  # print('*******')
  feedback_queries = list(feedback.keys())

  # feedback_query = feedback_queries[0]

  # feedback_query_tokenized = TOKENIZER.tokenize(
  #       feedback_query.translate(REMOVE_PUNCTUATION_TABLE)
  #       )

  vectorizer = TfidfVectorizer(tokenizer=tokenize_and_stem, stop_words='english')
  vectorizer.fit(review_list)

  feedback_queries = list(feedback.keys())

  similarity = cosine_similarity(vectorizer.transform([query]), 
                                vectorizer.transform(feedback_queries))
  
  #print(similarity)

  feedback_max_similarity = np.amax(similarity)

  #print(feedback_max_similarity)

  max_idx = np.argmax(similarity)

  feedback_queries[max_idx]

  pos_feedback_product_id = [idx for idx, feedback_value 
                        in feedback[feedback_queries[max_idx]] 
                        if feedback_value == 1.]

  counts = Counter(pos_feedback_product_id)

  pos_feedback_proportions = {
        doc_idx: count / sum(counts.values()) for doc_idx, count in counts.items()
  }

  pos_feedback_feature = {}
  for key in pos_feedback_proportions:
    pos_feedback_feature[key] = pos_feedback_proportions[key]*feedback_max_similarity
  
  for pid in pos_feedback_feature:
    if pid not in result_sim_df.values:
      result_sim_df = result_sim_df.append({'asin': pid, 'similarity':result_sim_df.get(pid)}, ignore_index=True)

  result_sim_df = result_sim_df[result_sim_df.groupby('asin')['similarity'].transform('max') == result_sim_df['similarity']]
  return result_sim_df